## Decorators

Un decorator in Python è una funzione (più in generale si può dire un **oggetto chiamabile**) che viene usato per modificare (decorare) il comportamento di una funzione o di una classe (e dei suoi oggetti).

Il decoratore prende in input la funzione (o la classe) e restituisce la funzione (o la classe) modificata

Abbiamo già utilizzato questa struttura (senza averla chiamata col nome appropriato) quando abbiamo usato @property, che un esempio notevole di decoratore

Ora vediamo la nozione in modo più diretto

### Ricordiamo che le funzioni sono first-class object

In [ ]:
def Fibonacci(n):
    f0 = 0
    f1 = 1
    for _ in range(n-1):
      f0, f1 = f1, f0+f1
    return f1

In [ ]:
Fibonacci(100)

In [ ]:
fibo = Fibonacci
fibo(10)

In [ ]:
def tabulate(sequence, f):
    return [f(x) for x in sequence]

In [ ]:
import math
tabulate([x*0.01 for x in range(157)], math.sin)

In [ ]:
def maketab(start, step, npoints):
    sequence = [start+i*step for i in range(npoints)]
    def tabulate(f):
        return [f(x) for x in sequence]
    return tabulate

In [ ]:
tabulate = maketab(0,0.01,157)

In [ ]:
tabulate(math.cos)

In [ ]:
def makePoly(*coefficients):
    def eval(x):
        value = 0.0
        for c in reversed(coefficients):
            value = value*x + c
        return value
    return eval

In [ ]:
p = makePoly(1,2,-1)

In [ ]:
for i in range(5):
    print(p(i))

### I decoratori mettono insieme le due "costruzioni": funzione come parametro e funzione come valore di ritorno, con una **sintassi specifica**

In [ ]:
def decoratore(f):
    def decorata(x):
        print('x\t f(x)')
        f(x)
    return decorata

In [ ]:
def g(x):
    print(x,'\t',math.sin(x))

In [ ]:
g(0.2)

In [ ]:
g = decoratore(g)

In [ ]:
gdec(0.2)

In [ ]:
@decoratore
def g(x):
    print(x,'\t',math.sin(x))

In [ ]:
g(0.2)

In [ ]:
def decoratore(f):
    def decorata(*x):
        print('x\t f(x)')
        for z in x:
            f(z)
    return decorata

In [ ]:
g(0.01,0.02,0.03)

In [ ]:
g([i/10 for i in range(10)])

### Tipici casi d'uso dei decoratori

La decorazione può utilmente "occuparsi" (in particolare, per la chiarezza del codice) dei casi particolari di input che devono essere controllati da un algoritmo.

In [ ]:
def roots2(a,b,c):
    delta = math.sqrt(b**2-4*a*c)
    x1 = (-b-delta)/(2*a)
    x2 = (-b+delta)/(2*a)
    return x1,x2

In [ ]:
roots2(0,-2,1)

In [ ]:
def special_cases(f):
    '''Per semplicità, tratta solo il caso in cui il coefficiente del termine quadratico è nullo'''
    def checker(a,b,c):
        if a==0:
            return -c/b
        else:
            return f(a,b,c)
    return checker

In [ ]:
@special_cases
def roots2(a,b,c):
    delta = math.sqrt(b**2-4*a*c)
    x1 = (-b-delta)/(2*a)
    x2 = (-b+delta)/(2*a)
    return x1,x2

In [ ]:
roots2(0,-2,1)

Un secondo caso di utilizzo è la raccolta di statistiche sull'uso di una funzione (tempo di esecuzione, numero di volte che è stata chiamata, ...)

In [ ]:
from time import time

In [ ]:
n = 1000000

In [ ]:
start_time = int(round(time() * 1000)) #tempo in millisecondi (trascorso a partire dal 1/1/1970
Fibonacci(n)
stop_time = int(round(time() * 1000))
elapsed_time = stop_time-start_time
print("Tempo di esecuzione di Fibonacci({}): {}ms".format(n, elapsed_time))

In [ ]:
def my_timer(f):
    def timedfun(n):
        start_time = int(round(time() * 1000))
        res = f(n)
        stop_time = int(round(time() * 1000))
        timedfun.time = stop_time-start_time
        return res
    return timedfun

@my_timer
def Fibonacci(n):
    f0 = 0
    f1 = 1
    for _ in range(n-1):
      f0, f1 = f1, f0+f1
    return f1

Fibonacci(n)
print("Tempo di esecuzione di Fibonacci({}): {}ms".format(n, Fibonacci.time))

In [ ]:
Fibonacci(n)
print("Tempo di esecuzione di Fibonacci({}): {}ms".format(n, Fibonacci.time))

In [ ]:
@my_timer
def insSort(A):
    n = len(A)
    for i in range(1,n):
        temp = A[i]
        j = i-1
        while j>=0 and A[j]>temp:
            A[j+1] = A[j]
            j = j-1
        A[j+1] = temp

In [ ]:
from random import randint

In [ ]:
n = 20000
A = [randint(1,100) for _ in range(n)]
#A

In [ ]:
insSort(A)
print("Tempo di esecuzione di insSort(A): {}ms".format(insSort.time))

In [ ]:
A

In [ ]:
def general_timer(f):
    def timedfun(*args, **kw):
        start_time = int(round(time() * 1000))
        res = f(*args, **kw)
        stop_time = int(round(time() * 1000))
        timedfun.time = stop_time-start_time
        print(1)
        return res
    return timedfun

In [ ]:
@general_timer
def insSort(A):
    n = len(A)
    for i in range(1,n):
        temp = A[i]
        j = i-1
        while j>=0 and A[j]>temp:
            A[j+1] = A[j]
            j = j-1
        A[j+1] = temp

In [ ]:
insSort(A)
print("Tempo di esecuzione di insSort(A): {}ms".format(insSort.time))

Occhio ai problemi con le funzioni ricorsive... (a cominciare dal valore di n)

In [ ]:
n = 30

In [ ]:
def Fibonacci_rec(n):
    if n == 0 or n == 1:
        return n
    return Fibonacci_rec(n-2)+Fibonacci_rec(n-1)

In [ ]:
start_time = int(round(time() * 1000)) #tempo in millisecondi (trascorso a partire dal 1/1/1970)
Fibonacci_rec(n)
stop_time = int(round(time() * 1000))
elapsed_time = stop_time-start_time
print("Tempo di esecuzione: {}ms".format(elapsed_time))

In [ ]:
@general_timer
def Fibonacci_rec(n):
    if n == 0 or n == 1:
        return n
    return Fibonacci_rec(n-2)+Fibonacci_rec(n-1)

In [ ]:
Fibonacci_rec(n)
print(Fibonacci_rec.time)

Una possibile soluzione (poco "elegante" ...).

In [ ]:
@general_timer
def Fibonacci_r(n):
    if n == 0 or n == 1:
        return n
    return Fibonacci_rec(n-2)+Fibonacci_rec(n-1)

In [ ]:
Fibonacci_r(n)
print(Fibonacci_r.time)

Anziché il tempo, si potrebbe voler misurare il numero di chiamate di una funzione

In [ ]:
def callcnt(f):
    def counter(*args, **kw):
        counter.calls += 1
        return f(*args, **kw)
    counter.calls = 0
    return counter

Questa volta non ci sono problemi con la ricorsione

In [ ]:
@callcnt
def Fibonacci_rec(n):
    if n == 1 or n == 2:
        return 1
    return Fibonacci_rec(n-2)+Fibonacci_rec(n-1)

In [ ]:
n = 30
Fibonacci_rec.calls = 0
v = Fibonacci_rec(n)
print("Il numero C(n) di chiamate ricorsive (con n = {}) è {}".format(n,Fibonacci_rec.calls))
print("Il valore di f({}) è {}".format(n,v))
print("Controllo, C(n) == 2*f(n)-1: {}".format(Fibonacci_rec.calls==2*v-1))

### Anche le classi possono essere utilizzate come decoratori

### Digressione. \__call\__: un interessante "metodo magico"

In [ ]:
class myrand:
    '''Un semplice generatore pseudo-casuale, da non usare per scopi crittografici...'''
    def __init__(self):
        self._a = 16807
        self._m = (1 << 31) - 1 #2^31 -1
        self._d = 1.0/self._m
        self._x = 1
        for i in range(10000):  # Per rendere i numeri "usati" (apparentemente) indipendenti dal seme iniziale
            self.__call__()
    
    def __call__(self):
        self._x = (self._a*self._x)%self._m
        return self._x*self._d

In [ ]:
r = myrand()

In [ ]:
for _ in range(10):
    print(r())

E' evidente che un tale generatore (del quale tutte le possibili istanze si comporterebbero alla stessa maniera) può essere realizzato in molti altri modi. Tuttavia l'esempio illustra il punto. Il metodo \__call\__ può essere chiamato col nome stesso della classe, il che rende l'uso di comprensione immediata. L'esempio illustra come sua possibile realizzare semplicemente una sorta di "funzione con stato", mediante classi e metodo \__call\__

Esempio: generazione di numeri distribuito con esponenziale negativa (di fondamentale utilità in esperimenti di simulazione)

In [ ]:
from math import log
class negexp:
    '''Utilizza un qualsiasi generatore uniforme e lo utilizza
       per calcolare "negative exponential deviate". l si interpreta
       come frequenza degli eventi (numero di eventi nell'unità di tempo)'''
    def __init__(self, l):
        self._lambda = l

    def __call__(self, fn):
        '''fn è il generatore uniforme'''
        def decorated():
            x = fn()
            return -log(1.0-x)/self._lambda
        return decorated

In [ ]:
@negexp(2)
def schedule():
    return random()

In [ ]:
schedule()

In [ ]:
negexp(2).__call__(random)()

In [ ]:
t = 0.0
count = 0
while t<100:
    t += schedule()
    count += 1
print(count)

In [ ]:
@negexp(10)
def schedulefast():
    return random()

In [ ]:
t = 0.0
count = 0
while t<100:
    t += schedulefast()
    count += 1
print(count)

In [ ]:
###### Progetto di un semplice simulatore con 1 coda e k serventi

from math import log
from random import random

import pdb

JOB_ARRIVAL = 0

class negexp:
    '''Utilizza un qualsiasi generatore uniforme e lo utilizza
       per calcolare "negative exponential deviate". l si interpreta
       come frequenza degli eventi (numero di eventi nell'unità di tempo)'''
    def __init__(self, l):
        self._lambda = l

    def __call__(self, fn):
        '''fn è il generatore uniforme'''
        def decorated():
            x = fn()
            return -log(1.0-x)/self._lambda
        return decorated

class job(int):
    '''Classe che definisce i job. Questi sono identificati semplicemente da numeri interi
    (ereditano da int) con alcune proprietà utilizzate per la gestione e per il calcolo delle
    statistiche:
    1) tempo di arrivo nel sistema
    2) tempo di inizio servizio
    3) tempo di fine servizio
    4) identificazione del server
    Il sistema è senza pre-emption (i job non vengono interrotti)
    '''
    
    __jobCnt = 1  # Contatore "automatico" per attribuire un id (int) ai job
    
    def __new__(cls, arrivalTime):
        j = super().__new__(cls, job.__jobCnt) # chiamata del costruttore della classe parent (int)
        j.__arrivalTime = arrivalTime          # tempo di arrivo del job
        job.__jobCnt += 1                      # incremento del contatore
        return j
    
    def __str__(self):
        j = super().__str__()
        return "job {}\n\tarrival time: {}\n\tservice time: {}\n\tend time: {}".format(j,\
                                                                                       self.arrivalTime,\
                                                                                       self.serviceTime,\
                                                                                       self.endTime)
    
    def inService(self, time, server, duration):
        '''Memorizza i tempi di inizio e fine servizio e il numero del servente'''
        self.__start = time
        self.__priority = time+duration
        self.__server = server
        
    @property
    def server(self):
        return self.__server
    
    @property
    def priority(self):
        return self.__priority
             
    @property
    def endTime(self):
        try:
            t = self.__priority
        except:
            print("Il job non è ancora completato")
            return
        return t
    
    @property
    def serviceTime(self):
        try:
            t = self.__start
        except:
            print("Il job è ancora in coda")
            return
        return t
    
    @property
    def arrivalTime(self):
        return self.__arrivalTime

class event(int):
    '''Classe che definisce gli eventi. Per questo simulatore sono rilevanti solo
    due tipi di evento:
    1) richiesta di un nuovo servizio (arrivo di un nuovo job),
    2) completamento del servizio di un job da parte di un server.
    La classe eredita da int e dunque gli eventi sono rappresentati come numeri interi.
    L'evento di arrivo di un nuovo job è rappresentato dalla costante JOB_ARRIVAL (che vale 0)
    mentre la terminazione di un job è rappresentata dal job stesso (che, si ricordi, è 
    un intero positivo). Ad ogni evento è associata una priorità, che è semplicemente il
    tempo al quale l'evento accade. Gli eventi schedulati (non ancora accaduti) sono memorizzati
    in una coda con priorità min-based, che è la struttura dati fondamentale del simulatore
    '''
    def __new__(cls, time, eventType):
        if type(eventType) == int:
            e = super().__new__(cls, eventType)  # Chiamata del costruttore della classe parent (int)
            e.__priority = time                  # Ogni evento ha un tipo e un tempo di occorrenza
        else:
            print('pippo')
            e = job             # Se l'evento è (la schedulazione di) un job, questo viene creato
        return e
    
    def __lt__(self, other):
        '''La precedenza di eventi è stabilita dalla priorità'''
        return self.priority < other.priority
    
    def eType(self):
        return type(self)
    
    @property
    def priority(self):
        return self.__priority
    
    @property
    def time(self):
        return self.__priority

class EmptyQueue(Exception):
    pass

class queue:
    '''Classe che implementa una coda semplice'''
    def __init__(self):
        self.__Q = []
    
    def enqueue(self, object):
        self.__Q.insert(0,object)
        
    def dequeue(self):
        if self.empty():
            raise EmptyQueue
        return self.__Q.pop()
    
    def empty(self):
        return self.__Q == []

class priority_queue:
    '''Implementa una coda con priorità (min-based) di oggetti arbitrari, che
       devono avere la sola proprietà di essere confrontabili con l'operatore <.
       L'attuale implementazione usa una semplice lista non ordinata. Questo
       rende efficiente l'inserzione ma non la ricerca e l'estrazione
       dell'oggetto con minimo valore di priorità'''
       
    def __init__(self, L=[]):
        if not L:
            self.__list = []
        else:
            self.__list = L

    def insert(self, obj):
        self.__list.append(obj)

    def is_empty(self):
        return self.__list == []

    def size(self):
        return len(self.__list)
    
    def minimum(self):
        if self.is_empty():
            raise EmptyQueue
        minindex = 0
        for i in range(1, len(self.__list)):
            if self.__list[i] < self.__list[minindex]:
                minindex = i
        return self.__list[minindex]
        
    def delete(self):
        if self.is_empty():
            raise EmptyQueue
        minindex = 0
        for i in range(1, len(self.__list)):
            if self.__list[i] < self.__list[minindex]:
                minindex = i
        x = self.__list[minindex]
        del self.__list[minindex]
        return x

class server:
    '''Classe che implementa i server'''
    
    def __init__(self, speed, system):
        @negexp(speed)
        def JobService():
            return random()
        
        self.__system = system
        self.__scheduler = JobService
        self.__busy = False
        
    def runJob(self, job, currentTime):
        if not self.busy:
            self.__busy = True
            self.__currentJob = job
            duration = self.__scheduler()
            job.inService(currentTime, self, duration)
    
    def endJob(self):
        self.__busy = False
        self.__system.jobTerminates(self, self.__currentJob)
          
    @property
    def busy(self):
        return self.__busy

class system:
    '''Classe che implementa il controllo dei k server e della coda'''
    def __init__(self, k, speed, pQ):
        self.__k = k
        self.__servers = [server(speed, self) for _ in range(k)]
        self.__Q = queue()
        self.__pQ = pQ
        self.__log = []
    
    def jobEnters(self, job, currentTime):
        s = self.available()
        if s:
            if self.__pQ.is_empty():
                t = currentTime
            else:
                t = max(currentTime,self.__pQ.minimum())
            s.runJob(job, t)
            self.__pQ.insert(job)
        else:
            self.__Q.enqueue(job)
    
    def available(self):
        for s in self.__servers:
            if not s.busy:
                return s
        return None

    def jobTerminates(self, server, job):
        self.__log.append(job)
        currentTime = job.endTime
        if not self.__Q.empty():
            self.jobEnters(self.__Q.dequeue(),currentTime)
            
    @property
    def log(self):
        return self.__log
    
def sim(numservers=1, speed=1, arrivalRate=1, simLen=2):
    
    @negexp(arrivalRate)
    def JobArrival():
        return random()
    
    Q = priority_queue()
    S = system(numservers, speed, Q)
    time = 0.0
    Q.insert(event(time,JOB_ARRIVAL))
    
    while time<simLen:
        e = Q.delete()
        time = e.priority
        if type(e) == job:
            e.server.endJob()
        else:
            j = job(time)
            S.jobEnters(j,time)
            nextArrival = JobArrival()
            Q.insert(event(time+nextArrival,JOB_ARRIVAL))
    
    for j in S.log:
        print(j)

In [ ]:
S = simulation(3,1,3)

In [ ]:
S._servers[0].run(0)

In [ ]:
j = [1,2,3]
s = server(j,schedule)
s.run(0)

In [ ]:
j